# Author: Andrew Yang

In [12]:
import pandas as pd
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.transformations.panel.compose import ColumnConcatenator
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sktime.datatypes import convert_to
from sktime.datatypes._panel._convert import from_2d_array_to_nested
from sklearn.preprocessing import LabelEncoder
import numpy as np
import os
# sys.path.append("..")
# List all CSV files in the data/raw directory
csv_files = [f for f in os.listdir('../data/raw') if f.endswith('.csv')]

# ,date,sender_id,bgl,bgl_date_millis,text,template,msg_type,affects_fob,affects_iob,dose_units,food_g,food_glycemic_index,dose_automatic,fp_bgl,message_basal_change,__typename,trend
# Define the columns to load
columns_to_load = ['date', 'bgl', 'msg_type', 'affects_fob', 'affects_iob','dose_units','food_g','food_glycemic_index']

# Load each CSV file into a DataFrame and store them in a dictionary
dataframes = {file: pd.read_csv(os.path.join('../data/raw', file), usecols=columns_to_load, parse_dates=['date']) for file in csv_files}

# Print the names of the loaded DataFrames
print("Loaded DataFrames:", list(dataframes.keys()))

Loaded DataFrames: ['500030_2024-07-01_2024-09-30.csv', '679372_2024-07-01_2024-09-30.csv']


In [30]:
df = dataframes['500030_2024-07-01_2024-09-30.csv']
df=df.fillna(0)
df['msg_type'] = df['msg_type'].replace(0, 'NULL')

In [31]:
df[pd.notna(df['food_g'])]

,date,bgl,msg_type,affects_fob,affects_iob,dose_units,food_g,food_glycemic_index,msg_type_encoded
0,2024-07-01 00:02:39-05:00,98.0,NULL,0,0,0.0,0.0,0.0,11
1,2024-07-01 00:07:39-05:00,100.0,NULL,0,0,0.0,0.0,0.0,11
2,2024-07-01 00:12:39-05:00,98.0,NULL,0,0,0.0,0.0,0.0,11
3,2024-07-01 00:17:39-05:00,94.0,NULL,0,0,0.0,0.0,0.0,11
4,2024-07-01 00:22:40-05:00,94.0,NULL,0,0,0.0,0.0,0.0,11
...,...,...,...,...,...,...,...,...,...
39622,2024-09-30 23:47:37-05:00,88.0,NULL,0,0,0.0,0.0,0.0,11
39623,2024-09-30 23:49:38-05:00,90.0,NULL,0,0,0.0,0.0,0.0,11
39624,2024-09-30 23:53:37-05:00,89.0,NULL,0,0,0.0,0.0,0.0,11
39625,2024-09-30 23:55:38-05:00,88.0,NULL,0,0,0.0,0.0,0.0,11


In [32]:
label_encoder = LabelEncoder()
df['msg_type_encoded'] = label_encoder.fit_transform(df['msg_type'])

In [33]:
X = df[['bgl']]  # Predictor variables
y = df['msg_type_encoded']  # Target variable
y_series = pd.Series(y, index=X.index)

In [34]:
X_train, X_test, y_train, y_test = temporal_train_test_split(X, y, test_size=0.2)
classifier = TimeSeriesForestClassifier(n_estimators=100, random_state=42)

In [35]:
classifier.fit(np.array(X_train), np.array(y_train))


/Users/andrewyang/.pyenv/versions/3.9.13/lib/python3.9/site-packages/sktime/utils/slope_and_trend.py:83: RuntimeWarning: invalid value encountered in divide
  return (np.mean(y * x, axis=axis) - x_mean * np.mean(y, axis=axis)) / (
/Users/andrewyang/.pyenv/versions/3.9.13/lib/python3.9/site-packages/sktime/utils/slope_and_trend.py:83: RuntimeWarning: invalid value encountered in divide
  return (np.mean(y * x, axis=axis) - x_mean * np.mean(y, axis=axis)) / (
/Users/andrewyang/.pyenv/versions/3.9.13/lib/python3.9/site-packages/sktime/utils/slope_and_trend.py:83: RuntimeWarning: invalid value encountered in divide
  return (np.mean(y * x, axis=axis) - x_mean * np.mean(y, axis=axis)) / (
/Users/andrewyang/.pyenv/versions/3.9.13/lib/python3.9/site-packages/sktime/utils/slope_and_trend.py:83: RuntimeWarning: invalid value encountered in divide
  return (np.mean(y * x, axis=axis) - x_mean * np.mean(y, axis=axis)) / (
/Users/andrewyang/.pyenv/versions/3.9.13/lib/python3.9/site-packages/sktime/

TimeSeriesForestClassifier(min_interval=1, n_estimators=100, random_state=42)

In [37]:
y_pred = classifier.predict(np.array(X_test))


/Users/andrewyang/.pyenv/versions/3.9.13/lib/python3.9/site-packages/sktime/utils/slope_and_trend.py:83: RuntimeWarning: invalid value encountered in divide
  return (np.mean(y * x, axis=axis) - x_mean * np.mean(y, axis=axis)) / (
/Users/andrewyang/.pyenv/versions/3.9.13/lib/python3.9/site-packages/sktime/utils/slope_and_trend.py:83: RuntimeWarning: invalid value encountered in divide
  return (np.mean(y * x, axis=axis) - x_mean * np.mean(y, axis=axis)) / (
/Users/andrewyang/.pyenv/versions/3.9.13/lib/python3.9/site-packages/sktime/utils/slope_and_trend.py:83: RuntimeWarning: invalid value encountered in divide
  return (np.mean(y * x, axis=axis) - x_mean * np.mean(y, axis=axis)) / (
/Users/andrewyang/.pyenv/versions/3.9.13/lib/python3.9/site-packages/sktime/utils/slope_and_trend.py:83: RuntimeWarning: invalid value encountered in divide
  return (np.mean(y * x, axis=axis) - x_mean * np.mean(y, axis=axis)) / (
/Users/andrewyang/.pyenv/versions/3.9.13/lib/python3.9/site-packages/sktime/

In [39]:
y_pred_decoded = label_encoder.inverse_transform(y_pred)
results = pd.DataFrame({
    'True Label': label_encoder.inverse_transform(y_test),
    'Predicted Label': y_pred_decoded
})

In [40]:
results

,True Label,Predicted Label
0,NULL,NULL
1,NULL,NULL
2,NULL,NULL
3,DOSE_INSULIN,NULL
4,NULL,NULL
...,...,...
7921,NULL,NULL
7922,NULL,NULL
7923,NULL,NULL
7924,NULL,NULL


In [44]:
from sklearn.metrics import accuracy_score, f1_score

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"Accuracy: {accuracy:.2f}")
print(f"F1: {f1:.2f}")

Accuracy: 0.96
F1: 0.94
